In [234]:
import pandas as pd
import numpy as np
import os
import glob
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import gc
import math
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Capacity Grouping

In [290]:
## Get Info about the WTP Capacity and water source
wtpc = pd.read_csv('Design Capacity WTP.csv')
wlb = ['small','medium','large']
breaks = [0,20,200,np.inf]
wtpc['ng'] = pd.cut(x=wtpc['Design Capacity (MLD)'], bins=breaks,labels=wlb)
wtpc['nng'] = pd.cut(x=wtpc['Design Capacity (MLD)'], bins=breaks)

In [291]:
wtpc

,Lembangan,Loji Rawatan Air,WTP Name,wtp,Water Source,Design Capacity (MLD),ng,nng
0,Sg Bernam,Kalumpang,Kalumpang,Kalumpang,River,6.7,small,"(0.0, 20.0]"
1,Sg Bernam,Sungai Selisek,Sungai Selisek,Sungai Selisek,River,1.3,small,"(0.0, 20.0]"
2,Sg Bernam,Bernam River Headworks,BRH,Bernam River Headworks (New),River,65.6,medium,"(20.0, 200.0]"
3,Sg Bernam,Bernam River Headworks,BRH,Bernam River Headworks (Old),River,65.6,medium,"(20.0, 200.0]"
4,Sg Bernam,Sungai Dusun,Sungai Dusun,Sungai Dusun,River,1.3,small,"(0.0, 20.0]"
5,Sg Bernam,Sungai Sireh,Sungai Sireh,Sungai Sireh,River,27.0,medium,"(20.0, 200.0]"
6,Sg Tengi,Sungai Tengi,Sungai Tengi,Sungai Tengi,River,1.3,small,"(0.0, 20.0]"
7,Sg Buloh,North Hummock,North Hummock,North Hummock,Dam,22.5,medium,"(20.0, 200.0]"
8,Sg Selangor,Batang Kali,Batang Kali,Batang Kali,River,20.3,medium,"(20.0, 200.0]"
9,Sg Selangor,Kuala Kubu Bharu,Kuala Kubu Bharu,Kuala Kubu Bharu,River,6.7,small,"(0.0, 20.0]"


## General Data Loading & Cleaning

### Water Quality (WQ) Related Data 

In [368]:
path = r'WQ\Refined'
db_dir = glob.glob(path + "/*.csv")

li = []

## Joining all csv file within the folder
for i in db_dir:
    df = pd.read_csv(i, index_col=None, header=0)
    li.append(df)
fdb = pd.concat(li, ignore_index=True)

In [356]:
## Drop unwanted column and fill na with big number 9999, then it will be removed when there is a change in data
fdb = fdb.drop(['Unnamed: 0'],axis=1)
fdb['Date'] = pd.to_datetime(fdb['Date'])
cl = fdb.loc[:, fdb.dtypes == object].columns.to_list()
cl.remove('wtp')
for i in cl:
    fdb[i] = fdb[i].fillna(9999)
    fdb[i] = fdb[i].apply(lambda x: str(x).replace(',',''))
fdb[cl] = fdb[cl].apply(pd.to_numeric)
db = fdb.copy()

c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13/10/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14/10/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/10/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\AIS06077\Anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '16/10/2022' in DD/MM/YYYY format. Provide form

In [370]:
fdb.to_csv('fdb.csv')

In [294]:
## Get all the relevant columns (Daily average for further processing)
avg = fdb.columns
l = ['Quarter','wtp','Month','Date']
avg = [x for x in avg if "avg" in x ] + l
db['Month'] = db['Date'].dt.month
db['Quarter'] = db['Date'].dt.quarter
dc = db[avg].copy()

## Drop due to data limitation (Only limited number of WTP do it on daily basis)
dc= dc.drop(['Aluminium_avg', 'Fluoride_avg'],axis=1)
avc = [x for x in dc.columns if "avg" in x ]

In [295]:
## Final Clean data for further processing
dc = pd.merge(dc,wtpc[['wtp','Water Source','Lembangan']],on='wtp',how='left')
dc = dc[(dc['Lembangan'].isin(['Sg Selangor'])) & (dc['Water Source']=='River')].copy()
dc = dc[dc['pH_avg']<14].copy()

## Chemical Dosage

#### Overall DB

In [300]:
path = r'CC\Chm_DB'
dcb_dir = glob.glob(path + "/*.csv")

mm = []

for i in dcb_dir:
    dbo = pd.read_csv(i, index_col=None, header=0)
    mm.append(dbo)

In [301]:
chdb = pd.concat(mm, ignore_index=True)
chdb = chdb.drop(['Unnamed: 0'],axis=1)
chdb = chdb[chdb['Date']!='MONTHLY'].reset_index(drop=True)
chdb['Date'] = chdb['Date'] + '-2021'
chdb['Date'] = pd.to_datetime(chdb['Date'])
chdb['Month'] = chdb['Date'].dt.month
chdb['Quarter'] = chdb['Date'].dt.quarter

In [303]:
chdb[chdb['wtp'] == 'Sungai Selangor Fasa 3 Stage 2']

,wtp,Date,Activated Carbon _avg,Aluminium Chlorohydrate _avg,Flopam AH 912 PWG _avg,Flopam FA 920 PWG _avg,Flopam FL 4520 PWG _avg,Flopam FO 4190 PWG _avg,Flopam FO 4290 PWG _avg,Hydrated Lime _avg,Hydrated Lime (Intermediate) _avg,Hydrated Lime (Post) _avg,Hydrated Lime (Powder) _avg,Hydrated Lime (Powder) (Intermediate) _avg,Hydrated Lime (Powder) (Post) _avg,Hydrated Lime (Powder) (Pre) _avg,Hydrated Lime (Pre) _avg,Kibbled Aluminium Sulphate _avg,Liquid Aluminium Sulphate _avg,Liquid Chlorine _avg,Liquid Chlorine (Intermediate) _avg,Liquid Chlorine (Post) _avg,Liquid Chlorine (Pre) _avg,Liquid Polyaluminium Chloride _avg,Microsand _avg,Modified Liquid Alum _avg,Polymer Tianfloc C3080 _avg,Potassium Permanganate _avg,Soda Ash _avg,Soda Ash (Post) _avg,Soda Ash (Pre) _avg,Sodium Aluminosilicate _avg,Sodium Fluoride _avg,Sodium Hypochlorite _avg,Sodium Silico Fluoride _avg,Superfloc C 591 PWG _avg,Activated Carbon _max,Aluminium Chlorohydrate _max,Flopam AH 912 PWG _max,Flopam FA 920 PWG _max,Flopam FL 4520 PWG _max,Flopam FO 4190 PWG _max,Flopam FO 4290 PWG _max,Hydrated Lime _max,Hydrated Lime (Intermediate) _max,Hydrated Lime (Post) _max,Hydrated Lime (Powder) _max,Hydrated Lime (Powder) (Intermediate) _max,Hydrated Lime (Powder) (Post) _max,Hydrated Lime (Powder) (Pre) _max,Hydrated Lime (Pre) _max,Kibbled Aluminium Sulphate _max,Liquid Aluminium Sulphate _max,Liquid Chlorine _max,Liquid Chlorine (Intermediate) _max,Liquid Chlorine (Post) _max,Liquid Chlorine (Pre) _max,Liquid Polyaluminium Chloride _max,Microsand _max,Modified Liquid Alum _max,Polymer Tianfloc C3080 _max,Potassium Permanganate _max,Soda Ash _max,Soda Ash (Post) _max,Soda Ash (Pre) _max,Sodium Aluminosilicate _max,Sodium Fluoride _max,Sodium Hypochlorite _max,Sodium Silico Fluoride _max,Superfloc C 591 PWG _max,Activated Carbon _min,Aluminium Chlorohydrate _min,Flopam AH 912 PWG _min,Flopam FA 920 PWG _min,Flopam FL 4520 PWG _min,Flopam FO 4190 PWG _min,Flopam FO 4290 PWG _min,Hydrated Lime _min,Hydrated Lime (Intermediate) _min,Hydrated Lime (Post) _min,Hydrated Lime (Powder) _min,Hydrated Lime (Powder) (Intermediate) _min,Hydrated Lime (Powder) (Post) _min,Hydrated Lime (Powder) (Pre) _min,Hydrated Lime (Pre) _min,Kibbled Aluminium Sulphate _min,Liquid Aluminium Sulphate _min,Liquid Chlorine _min,Liquid Chlorine (Intermediate) _min,Liquid Chlorine (Post) _min,Liquid Chlorine (Pre) _min,Liquid Polyaluminium Chloride _min,Microsand _min,Modified Liquid Alum _min,Polymer Tianfloc C3080 _min,Potassium Permanganate _min,Soda Ash _min,Soda Ash (Post) _min,Soda Ash (Pre) _min,Sodium Aluminosilicate _min,Sodium Fluoride _min,Sodium Hypochlorite _min,Sodium Silico Fluoride _min,Superfloc C 591 PWG _min,Polyelectrolyte AN 910 PWG _avg,Polyelectrolyte AN 910 PWG _max,Polyelectrolyte AN 910 PWG _min,Liquid Polyaluminium Chloride (mg/L) _avg,Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L) _avg,Hydrated Lime (Post) (mg/L) _avg,Liquid Chlorine (Post) (mg/L) _avg,Sodium Silico Fluoride (mg/L) _avg,Liquid Chlorine (Pre) (mg/L) _avg,Hydrated Lime (Pre) (mg/L) _avg,Month,Quarter
27275,Sungai Selangor Fasa 3 Stage 2,2022-01-01 20:21:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.94,0.068,5.72,4.24,0.45,1.08,1.22,1,1
27276,Sungai Selangor Fasa 3 Stage 2,2022-02-01 20:21:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

In [304]:
FC = chdb.columns
fFC = [x for x in FC if "avg" in x]
rcol = ['wtp','Date','Month','Quarter']
xxdb = chdb[fFC+rcol].copy()

In [305]:
FC

Index(['wtp', 'Date', 'Activated Carbon _avg', 'Aluminium Chlorohydrate _avg',
       'Flopam AH 912 PWG _avg', 'Flopam FA 920 PWG _avg',
       'Flopam FL 4520 PWG _avg', 'Flopam FO 4190 PWG _avg',
       'Flopam FO 4290 PWG _avg', 'Hydrated Lime _avg',
       ...
       'Polyelectrolyte AN 910 PWG _min',
       'Liquid Polyaluminium Chloride (mg/L) _avg',
       'Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L) _avg',
       'Hydrated Lime (Post) (mg/L) _avg',
       'Liquid Chlorine (Post) (mg/L) _avg',
       'Sodium Silico Fluoride (mg/L) _avg',
       'Liquid Chlorine (Pre) (mg/L) _avg', 'Hydrated Lime (Pre) (mg/L) _avg',
       'Month', 'Quarter'],
      dtype='object', length=116)

In [306]:
chdb[fFC+rcol]

,Activated Carbon _avg,Aluminium Chlorohydrate _avg,Flopam AH 912 PWG _avg,Flopam FA 920 PWG _avg,Flopam FL 4520 PWG _avg,Flopam FO 4190 PWG _avg,Flopam FO 4290 PWG _avg,Hydrated Lime _avg,Hydrated Lime (Intermediate) _avg,Hydrated Lime (Post) _avg,Hydrated Lime (Powder) _avg,Hydrated Lime (Powder) (Intermediate) _avg,Hydrated Lime (Powder) (Post) _avg,Hydrated Lime (Powder) (Pre) _avg,Hydrated Lime (Pre) _avg,Kibbled Aluminium Sulphate _avg,Liquid Aluminium Sulphate _avg,Liquid Chlorine _avg,Liquid Chlorine (Intermediate) _avg,Liquid Chlorine (Post) _avg,Liquid Chlorine (Pre) _avg,Liquid Polyaluminium Chloride _avg,Microsand _avg,Modified Liquid Alum _avg,Polymer Tianfloc C3080 _avg,Potassium Permanganate _avg,Soda Ash _avg,Soda Ash (Post) _avg,Soda Ash (Pre) _avg,Sodium Aluminosilicate _avg,Sodium Fluoride _avg,Sodium Hypochlorite _avg,Sodium Silico Fluoride _avg,Superfloc C 591 PWG _avg,Polyelectrolyte AN 910 PWG _avg,Liquid Polyaluminium Chloride (mg/L) _avg,Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L) _avg,Hydrated Lime (Post) (mg/L) _avg,Liquid Chlorine (Post) (mg/L) _avg,Sodium Silico Fluoride (mg/L) _avg,Liquid Chlorine (Pre) (mg/L) _avg,Hydrated Lime (Pre) (mg/L) _avg,wtp,Date,Month,Quarter
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.67,NaN,1.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.67,NaN,NaN,NaN,NaN,NaN,0.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ampang Intake,2021-01-01 20:21:00,1,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.33,NaN,1.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.33,NaN,NaN,NaN,NaN,NaN,0.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ampang Intake,2021-01-10 20:21:00,1,1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.51,NaN,1.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.36,NaN,NaN,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ampang Intake,2021-01-11 20:21:00,1,1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.98,NaN,1.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.25,NaN,NaN,NaN,NaN,NaN,0.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ampang Intake,2021-01-12 20:21:00,1,1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.61,NaN,1.31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.61,NaN,NaN,NaN,NaN,NaN,0.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ampang Intake,2021-01-02 20:21:00,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.64,0.068,8.79,3.61,0.55,0.62,2.11,Sungai Selangor Fasa 3 Stage 2,2022-12-27 20:21:00,12,4
28366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.97,0.055,8.94,3.56,0.53,0.57,1.90,Sungai Selangor Fasa 3 Stage 2,2022-12-28 20:21:00,12,4
28367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.84,0.058,8.82,3.96,0.57,1.08,2.15,Sungai Selangor Fasa 3 Stage 2,2022-12-29 20:21:00,12,4
28368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.05,0.098,8.73,3.65,0.58,0.80,2.11,Sungai Selangor Fasa 3 Stage 2,2022-12-30 20:21:00,12,4


In [307]:
new = [x for x in FC if "avg" in x]
new = [x.replace(" _avg","") for x in new]
new = [x.replace("_avg","") for x in new]

### Regrouping Chemical Product to it's own purposes

In [308]:
xdic = dict(zip(fFC,new))
xxdb = xxdb.rename(columns=xdic)
xxdb = pd.merge(xxdb,wtpc[['wtp','Water Source','Lembangan']],on='wtp',how='left')
x1db = xxdb[(xxdb['Lembangan']=='Sg Selangor') & (xxdb['Water Source']=='River')].copy()

In [309]:
x1db

,Activated Carbon,Aluminium Chlorohydrate,Flopam AH 912 PWG,Flopam FA 920 PWG,Flopam FL 4520 PWG,Flopam FO 4190 PWG,Flopam FO 4290 PWG,Hydrated Lime,Hydrated Lime (Intermediate),Hydrated Lime (Post),Hydrated Lime (Powder),Hydrated Lime (Powder) (Intermediate),Hydrated Lime (Powder) (Post),Hydrated Lime (Powder) (Pre),Hydrated Lime (Pre),Kibbled Aluminium Sulphate,Liquid Aluminium Sulphate,Liquid Chlorine,Liquid Chlorine (Intermediate),Liquid Chlorine (Post),Liquid Chlorine (Pre),Liquid Polyaluminium Chloride,Microsand,Modified Liquid Alum,Polymer Tianfloc C3080,Potassium Permanganate,Soda Ash,Soda Ash (Post),Soda Ash (Pre),Sodium Aluminosilicate,Sodium Fluoride,Sodium Hypochlorite,Sodium Silico Fluoride,Superfloc C 591 PWG,Polyelectrolyte AN 910 PWG,Liquid Polyaluminium Chloride (mg/L),Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L),Hydrated Lime (Post) (mg/L),Liquid Chlorine (Post) (mg/L),Sodium Silico Fluoride (mg/L),Liquid Chlorine (Pre) (mg/L),Hydrated Lime (Pre) (mg/L),wtp,Date,Month,Quarter,Water Source,Lembangan
365,NaN,NaN,NaN,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.16,NaN,2.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.9,0.00,NaN,NaN,NaN,0.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Batang Kali,2021-01-01 20:21:00,1,1,River,Sg Selangor
366,NaN,NaN,NaN,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.44,NaN,2.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.04,NaN,NaN,NaN,0.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Batang Kali,2021-01-10 20:21:00,1,1,River,Sg Selangor
367,NaN,NaN,NaN,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.23,NaN,2.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.01,NaN,NaN,NaN,0.51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Batang Kali,2021-01-11 20:21:00,1,1,River,Sg Selangor
368,NaN,NaN,NaN,0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.48,NaN,2.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,2.20,NaN,NaN,NaN,0.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Batang Kali,2021-01-12 20:21:00,1,1,River,Sg Selangor
369,NaN,NaN,NaN,0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.37,NaN,2.14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.88,NaN,NaN,NaN,0.47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Batang Kali,2021-01-02 20:21:00,1,1,River,Sg Selangor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.64,0.068,8.79,3.61,0.55,0.62,2.11,Sungai Selangor Fasa 3 Stage 2,2022-12-27 20:21:00,12,4,River,Sg Selangor
28366,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44.97,0.055,8.94,3.56,0.53,0.57,1.90,Sungai Selangor Fasa 3 Stage 2,2022-12-28 20:21:00,12,4,River,Sg Selangor
28367,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.84,0.058,8.82,3.96,0.57,1.08,2.15,Sungai Selangor Fasa 3 Stage 2,2022-12-29 20:21:00,12,4,River,Sg Selangor
28368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38.05,0.098,8.73,3.65,0.58,0.80,2.11,Sungai Selangor Fasa 3 Stage 2,2022-12-30 20:21:00,12,4,River,Sg Selangor


In [313]:
appl = x1db.groupby(['wtp']).mean().reset_index()
wtpn = appl.wtp.unique()
number = dict({'wtp':[],'Chem':[]})
for i in wtpn:
    number['wtp'].append(i)
    print(i)
    jde = appl[appl['wtp']==i].drop(['wtp'],axis=1)
    jde
    jde = jde.dropna(axis=1).columns
    jde
    number['Chem'].append(jde)

Batang Kali


Index(['Flopam FA 920 PWG', 'Kibbled Aluminium Sulphate', 'Liquid Chlorine',
       'Soda Ash (Post)', 'Soda Ash (Pre)', 'Sodium Silico Fluoride', 'Month',
       'Quarter'],
      dtype='object')

Kuala Kubu Bharu


Index(['Hydrated Lime', 'Kibbled Aluminium Sulphate', 'Liquid Chlorine',
       'Soda Ash', 'Sodium Silico Fluoride', 'Month', 'Quarter'],
      dtype='object')

Rantau Panjang New


...

Index(['Flopam FO 4290 PWG', 'Hydrated Lime (Powder) (Post)',
       'Hydrated Lime (Powder) (Pre)', 'Liquid Aluminium Sulphate',
       'Liquid Chlorine (Post)', 'Liquid Chlorine (Pre)', 'Microsand',
       'Sodium Silico Fluoride', 'Month', 'Quarter'],
      dtype='object')

Sungai Selangor Fasa 3 Stage 1


Index(['Liquid Polyaluminium Chloride (mg/L)',
       'Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)',
       'Hydrated Lime (Post) (mg/L)', 'Liquid Chlorine (Post) (mg/L)',
       'Liquid Chlorine (Pre) (mg/L)', 'Hydrated Lime (Pre) (mg/L)', 'Month',
       'Quarter'],
      dtype='object')

Sungai Selangor Fasa 3 Stage 2


Index(['Liquid Polyaluminium Chloride (mg/L)',
       'Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)',
       'Hydrated Lime (Post) (mg/L)', 'Liquid Chlorine (Post) (mg/L)',
       'Liquid Chlorine (Pre) (mg/L)', 'Hydrated Lime (Pre) (mg/L)', 'Month',
       'Quarter'],
      dtype='object')

In [324]:
appl = x1db.groupby(['wtp']).max()#.reset_index()
#appl[appl['wtp']=='Sungai Selangor Fasa 3 Phase 1'].columns
appl.columns

Index(['Activated Carbon', 'Aluminium Chlorohydrate', 'Flopam AH 912 PWG',
       'Flopam FA 920 PWG', 'Flopam FL 4520 PWG', 'Flopam FO 4190 PWG',
       'Flopam FO 4290 PWG', 'Hydrated Lime', 'Hydrated Lime (Intermediate)',
       'Hydrated Lime (Post)', 'Hydrated Lime (Powder)',
       'Hydrated Lime (Powder) (Intermediate)',
       'Hydrated Lime (Powder) (Post)', 'Hydrated Lime (Powder) (Pre)',
       'Hydrated Lime (Pre)', 'Kibbled Aluminium Sulphate',
       'Liquid Aluminium Sulphate', 'Liquid Chlorine',
       'Liquid Chlorine (Intermediate)', 'Liquid Chlorine (Post)',
       'Liquid Chlorine (Pre)', 'Liquid Polyaluminium Chloride', 'Microsand',
       'Modified Liquid Alum', 'Polymer Tianfloc C3080',
       'Potassium Permanganate', 'Soda Ash', 'Soda Ash (Post)',
       'Soda Ash (Pre)', 'Sodium Aluminosilicate', 'Sodium Fluoride',
       'Sodium Hypochlorite', 'Sodium Silico Fluoride', 'Superfloc C 591 PWG',
       'Polyelectrolyte AN 910 PWG', 'Liquid Polyaluminium Chloride 

In [320]:
x1db[x1db['wtp']==i].columns

Index(['Activated Carbon', 'Aluminium Chlorohydrate', 'Flopam AH 912 PWG',
       'Flopam FA 920 PWG', 'Flopam FL 4520 PWG', 'Flopam FO 4190 PWG',
       'Flopam FO 4290 PWG', 'Hydrated Lime', 'Hydrated Lime (Intermediate)',
       'Hydrated Lime (Post)', 'Hydrated Lime (Powder)',
       'Hydrated Lime (Powder) (Intermediate)',
       'Hydrated Lime (Powder) (Post)', 'Hydrated Lime (Powder) (Pre)',
       'Hydrated Lime (Pre)', 'Kibbled Aluminium Sulphate',
       'Liquid Aluminium Sulphate', 'Liquid Chlorine',
       'Liquid Chlorine (Intermediate)', 'Liquid Chlorine (Post)',
       'Liquid Chlorine (Pre)', 'Liquid Polyaluminium Chloride', 'Microsand',
       'Modified Liquid Alum', 'Polymer Tianfloc C3080',
       'Potassium Permanganate', 'Soda Ash', 'Soda Ash (Post)',
       'Soda Ash (Pre)', 'Sodium Aluminosilicate', 'Sodium Fluoride',
       'Sodium Hypochlorite', 'Sodium Silico Fluoride', 'Superfloc C 591 PWG',
       'Polyelectrolyte AN 910 PWG', 'Liquid Polyaluminium Chloride 

In [339]:
appl = x1db.groupby(['wtp']).mean().reset_index()
wtpn = appl.wtp.unique()
number = dict({'wtp':[],'Chem':[]})
for i in wtpn:
    number['wtp'].append(i)
    jde = appl[appl['wtp']==i].drop(['wtp'],axis=1)
    jde = jde.dropna(axis=1).columns
    number['Chem'].append(jde)
cgr = pd.read_csv("C.Grouping.csv")
Chdict = dict(zip(cgr['Chemical Name'].values,cgr['Correct Grouping']))
Chdict_2 = {}

for key in Chdict:
    Chdict_2[key] = key

df22=pd.DataFrame(number)
df22=df22.assign(Chem=df22.Chem.map(','.join))
df22=df22.set_index(['wtp']).apply(lambda x: x.str.split(',').explode()).reset_index()
df22['Purpose']=df22['Chem'].apply(lambda x: Chdict_2.get(x))

In [340]:
Chdict_2

{'Activated Carbon': 'Activated Carbon',
 'Activated Carbon (25kg / bag)': 'Activated Carbon (25kg / bag)',
 'Activated Carbon (500kg / 1000kg per bag)': 'Activated Carbon (500kg / 1000kg per bag)',
 'Aluminium Chlorohydrate': 'Aluminium Chlorohydrate',
 'Aluminium Chlorohydrate (bulk)': 'Aluminium Chlorohydrate (bulk)',
 'Chloride of Lime (50kg / drum)': 'Chloride of Lime (50kg / drum)',
 'Citric Acid (25kg / carboy)': 'Citric Acid (25kg / carboy)',
 'Flopam AH 912 PWG': 'Flopam AH 912 PWG',
 'Flopam AH 912 PWG (25kg / bag)': 'Flopam AH 912 PWG (25kg / bag)',
 'Flopam FA 920 PWG': 'Flopam FA 920 PWG',
 'Flopam FA 920 PWG (25kg / bag)': 'Flopam FA 920 PWG (25kg / bag)',
 'Flopam FL 4520 PWG': 'Flopam FL 4520 PWG',
 'Flopam FL 4520 PWG (210kg / drum)': 'Flopam FL 4520 PWG (210kg / drum)',
 'Flopam FO 4190 PWG': 'Flopam FO 4190 PWG',
 'Flopam FO 4190 PWG (25kg / bag)': 'Flopam FO 4190 PWG (25kg / bag)',
 'Flopam FO 4290 PWG': 'Flopam FO 4290 PWG',
 'Flopam FO 4290 PWG (25kg / bag)': 'Flo

In [341]:
lis = df22['Chem'].unique()
pre_lis = [x for x in lis if "Pre" in x]
pos_lis = [x for x in lis if "Post" in x]
int_lis = [x for x in lis if "Intermediate" in x]
total_lis = pre_lis + pos_lis + int_lis

In [342]:
sd = ['Soda Ash (Pre)','Soda Ash (Post)']
hl = ['Hydrated Lime (Powder) (Pre)','Hydrated Lime (Powder) (Post)','Hydrated Lime (Pre) (mg/L)','Hydrated Lime (Post) (mg/L)']
lc = ['Liquid Chlorine (Pre)','Liquid Chlorine (Post)','Liquid Chlorine (Pre) (mg/L)','Liquid Chlorine (Post) (mg/L)']

In [343]:
x1db[total_lis] = x1db[total_lis].fillna(0)
x1db['Soda_Ash_T'] = x1db[sd].sum(axis=1)
x1db['Hydrated_Lime_Pow_T'] = x1db[hl].sum(axis=1)
x1db['Liquid_Chlorine_T'] = x1db[lc].sum(axis=1)

In [344]:
li = [dict(zip(sd,['Soda_Ash_T' for i in range(len(sd))])),
      dict(zip(lc,['Liquid_Chlorine_T' for i in range(len(lc))])),
      dict(zip(hl,['Hydrated_Lime_Pow_T' for i in range(len(hl))]))]

In [345]:
d = {}
for dictionary in li:
    for k, v in dictionary.items():
        d[k] = v

In [346]:
df22['Chem'] = df22['Chem'].apply(lambda x: d.get(x,x))
df22 = df22.drop_duplicates(subset=['wtp','Chem'])
df22 = df22.drop_duplicates(subset=['wtp','Purpose'],keep='first')
tt = x1db['wtp'].unique()
trdf = df22.copy()
Chdict['Hydrated_Lime_Pow_T'] = 'pH Adjuster'
Chdict['Soda_Ash_T'] = 'pH Adjuster'
Chdict['Liquid_Chlorine_T'] = 'Disinfectant'

In [354]:
df22

,wtp,Chem,Purpose
0,Batang Kali,Flopam FA 920 PWG,Flopam FA 920 PWG
1,Batang Kali,Hydrated_Lime_Pow_T,Hydrated Lime (Powder) (Post)
3,Batang Kali,Kibbled Aluminium Sulphate,Kibbled Aluminium Sulphate
4,Batang Kali,Liquid Chlorine,Liquid Chlorine
5,Batang Kali,Liquid Chlorine (Intermediate),Liquid Chlorine (Intermediate)
...,...,...,...
209,Sungai Selangor Fasa 3 Stage 2,Liquid_Chlorine_T,Liquid Chlorine (Post)
211,Sungai Selangor Fasa 3 Stage 2,Soda_Ash_T,Soda Ash (Post)
213,Sungai Selangor Fasa 3 Stage 2,Liquid Polyaluminium Chloride (mg/L),Liquid Polyaluminium Chloride (mg/L)
214,Sungai Selangor Fasa 3 Stage 2,Raw Water Polyelectrolyte 4290 PWG / 4190 PWG ...,Raw Water Polyelectrolyte 4290 PWG / 4190 PWG ...


In [347]:
jam = pd.DataFrame()
x10db = x1db.copy()

In [348]:
#x10db.to_csv('CP_data\ChemPare_SgSel_6_2022.csv')

In [349]:
print(x1db[x1db['wtp'] == 'Sungai Selangor Fasa 3 Stage 1'])

       Activated Carbon  Aluminium Chlorohydrate  Flopam AH 912 PWG  \
26910               NaN                      NaN                NaN   
26911               NaN                      NaN                NaN   
26912               NaN                      NaN                NaN   
26913               NaN                      NaN                NaN   
26914               NaN                      NaN                NaN   
...                 ...                      ...                ...   
28000               NaN                      NaN                NaN   
28001               NaN                      NaN                NaN   
28002               NaN                      NaN                NaN   
28003               NaN                      NaN                NaN   
28004               NaN                      NaN                NaN   

       Flopam FA 920 PWG  Flopam FL 4520 PWG  Flopam FO 4190 PWG  \
26910                NaN                 NaN                 NaN   
26911      

In [350]:
x1db.columns

Index(['Activated Carbon', 'Aluminium Chlorohydrate', 'Flopam AH 912 PWG',
       'Flopam FA 920 PWG', 'Flopam FL 4520 PWG', 'Flopam FO 4190 PWG',
       'Flopam FO 4290 PWG', 'Hydrated Lime', 'Hydrated Lime (Intermediate)',
       'Hydrated Lime (Post)', 'Hydrated Lime (Powder)',
       'Hydrated Lime (Powder) (Intermediate)',
       'Hydrated Lime (Powder) (Post)', 'Hydrated Lime (Powder) (Pre)',
       'Hydrated Lime (Pre)', 'Kibbled Aluminium Sulphate',
       'Liquid Aluminium Sulphate', 'Liquid Chlorine',
       'Liquid Chlorine (Intermediate)', 'Liquid Chlorine (Post)',
       'Liquid Chlorine (Pre)', 'Liquid Polyaluminium Chloride', 'Microsand',
       'Modified Liquid Alum', 'Polymer Tianfloc C3080',
       'Potassium Permanganate', 'Soda Ash', 'Soda Ash (Post)',
       'Soda Ash (Pre)', 'Sodium Aluminosilicate', 'Sodium Fluoride',
       'Sodium Hypochlorite', 'Sodium Silico Fluoride', 'Superfloc C 591 PWG',
       'Polyelectrolyte AN 910 PWG', 'Liquid Polyaluminium Chloride 

In [351]:
df22[df22['wtp'] == 'Sungai Selangor Fasa 3 Stage 1']

,wtp,Chem,Purpose
188,Sungai Selangor Fasa 3 Stage 1,Hydrated_Lime_Pow_T,Hydrated Lime (Powder) (Post)
190,Sungai Selangor Fasa 3 Stage 1,Liquid Chlorine (Intermediate),Liquid Chlorine (Intermediate)
191,Sungai Selangor Fasa 3 Stage 1,Liquid_Chlorine_T,Liquid Chlorine (Post)
193,Sungai Selangor Fasa 3 Stage 1,Soda_Ash_T,Soda Ash (Post)
195,Sungai Selangor Fasa 3 Stage 1,Liquid Polyaluminium Chloride (mg/L),Liquid Polyaluminium Chloride (mg/L)
196,Sungai Selangor Fasa 3 Stage 1,Raw Water Polyelectrolyte 4290 PWG / 4190 PWG ...,Raw Water Polyelectrolyte 4290 PWG / 4190 PWG ...
201,Sungai Selangor Fasa 3 Stage 1,Month,None


#### Chemical Product Normalization

In [352]:
onew = 0
for i in tt:
    
    c2t = trdf[trdf['wtp']==i]['Chem'].unique()
    print(i)
    print(c2t)
    c2t = np.append(c2t,np.array(['Date','wtp']))
    c2t
    op = x10db[x10db['wtp']==i][c2t].copy()
    op = op.rename(columns=Chdict_2)
    
    if onew == 0:
        jam = jam.append(op)
    else:
        jam = pd.concat([jam, op], ignore_index=False)
    
    onew = onew+1
    
    del c2t, op
    gc.collect()

Batang Kali
['Flopam FA 920 PWG' 'Hydrated_Lime_Pow_T' 'Kibbled Aluminium Sulphate'
 'Liquid Chlorine' 'Liquid Chlorine (Intermediate)' 'Liquid_Chlorine_T'
 'Soda_Ash_T' 'Sodium Silico Fluoride' 'Month']


array(['Flopam FA 920 PWG', 'Hydrated_Lime_Pow_T',
       'Kibbled Aluminium Sulphate', 'Liquid Chlorine',
       'Liquid Chlorine (Intermediate)', 'Liquid_Chlorine_T',
       'Soda_Ash_T', 'Sodium Silico Fluoride', 'Month', 'Date', 'wtp'],
      dtype=object)

C:\Users\AIS06077\AppData\Local\Temp\ipykernel_4960\1331729719.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  jam = jam.append(op)


69

Kuala Kubu Bharu
['Hydrated Lime' 'Hydrated_Lime_Pow_T' 'Kibbled Aluminium Sulphate'
 'Liquid Chlorine' 'Liquid Chlorine (Intermediate)' 'Liquid_Chlorine_T'
 'Soda Ash' 'Soda_Ash_T' 'Sodium Silico Fluoride' 'Month']


...

array(['Hydrated_Lime_Pow_T', 'Liquid Chlorine (Intermediate)',
       'Liquid_Chlorine_T', 'Soda_Ash_T',
       'Liquid Polyaluminium Chloride (mg/L)',
       'Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)',
       'Month', 'Date', 'wtp'], dtype=object)

0

Sungai Selangor Fasa 3 Stage 2
['Hydrated_Lime_Pow_T' 'Liquid Chlorine (Intermediate)'
 'Liquid_Chlorine_T' 'Soda_Ash_T' 'Liquid Polyaluminium Chloride (mg/L)'
 'Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)' 'Month']


array(['Hydrated_Lime_Pow_T', 'Liquid Chlorine (Intermediate)',
       'Liquid_Chlorine_T', 'Soda_Ash_T',
       'Liquid Polyaluminium Chloride (mg/L)',
       'Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)',
       'Month', 'Date', 'wtp'], dtype=object)

0

In [353]:
trdf[trdf['wtp']==i]['Chem']

206                                  Hydrated_Lime_Pow_T
208                       Liquid Chlorine (Intermediate)
209                                    Liquid_Chlorine_T
211                                           Soda_Ash_T
213                 Liquid Polyaluminium Chloride (mg/L)
214    Raw Water Polyelectrolyte 4290 PWG / 4190 PWG ...
219                                                Month
Name: Chem, dtype: object

### Final Merge Data

In [263]:
wqdb1 = dc[dc['Lembangan']=='Sg Selangor'][['wtp', 'pH_avg', 'Date', 'Turbidity_avg', 'Color_avg',
       'Iron_avg', 'Ammonia_avg', 'Manganese_avg']].copy()
wqdb1['Date'] = pd.to_datetime(wqdb1['Date']).dt.date
jam['Date'] = pd.to_datetime(jam['Date']).dt.date
## Dropping Polyelectrolyte because project does not focus study on these product (Low usage & only certain WTP using it)
FFDB = pd.merge(wqdb1.copy(),jam.copy(),on=['wtp','Date'],how='left')


In [264]:
dc[dc['wtp'] == 'Sungai Selangor Fasa 3 Stage 2']['Lembangan'].unique()

array(['Sg Selangor'], dtype=object)

In [265]:
FFDB.to_csv('CP_data\ChemPare_SgSel_2021_NEW_2022.csv')
jam4 = FFDB.copy().fillna(0)

In [266]:
jam4[jam4['wtp'] == 'Sungai Selangor Fasa 3 Stage 1']

,wtp,pH_avg,Date,Turbidity_avg,Color_avg,Iron_avg,Ammonia_avg,Manganese_avg,Flopam FA 920 PWG,Kibbled Aluminium Sulphate,Liquid Chlorine,Soda_Ash_T,Sodium Silico Fluoride,Month,Hydrated Lime,Soda Ash,Liquid Polyaluminium Chloride,Sodium Aluminosilicate,Flopam FO 4290 PWG,Hydrated_Lime_Pow_T,Liquid Aluminium Sulphate,Liquid_Chlorine_T,Microsand,Flopam FO 4190 PWG,Liquid Chlorine (Intermediate),Liquid Polyaluminium Chloride (mg/L),Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L),Polyelectrolyte AN 910 PWG
5226,Sungai Selangor Fasa 3 Stage 1,5.55,2022-01-01,199.08,31.33,1.96,0.17,0.37,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,0.0,9.55,0.0,6.21,0.0,0.0,0.0,30.13,0.044,0.0
5227,Sungai Selangor Fasa 3 Stage 1,5.72,2022-02-01,275.25,29.50,2.05,0.18,0.40,0.0,0.0,0.0,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,9.32,0.0,5.78,0.0,0.0,0.0,31.24,0.062,0.0
5228,Sungai Selangor Fasa 3 Stage 1,5.71,2022-03-01,168.79,30.58,1.70,0.10,0.29,0.0,0.0,0.0,0.0,0.0,3,0.0,0.0,0.0,0.0,0.0,9.31,0.0,5.33,0.0,0.0,0.0,32.23,0.073,0.0
5229,Sungai Selangor Fasa 3 Stage 1,5.53,2022-04-01,127.17,39.50,2.01,0.18,0.36,0.0,0.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,0.0,9.37,0.0,5.35,0.0,0.0,0.0,30.58,0.065,0.0
5230,Sungai Selangor Fasa 3 Stage 1,5.79,2022-05-01,81.06,42.67,1.20,0.12,0.17,0.0,0.0,0.0,0.0,0.0,5,0.0,0.0,0.0,0.0,0.0,9.10,0.0,6.49,0.0,0.0,0.0,35.52,0.062,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5586,Sungai Selangor Fasa 3 Stage 1,6.23,2022-12-27,74.65,40.67,1.94,0.13,0.25,0.0,0.0,0.0,0.0,0.0,12,0.0,0.0,0.0,0.0,0.0,11.13,0.0,5.00,0.0,0.0,0.0,49.58,0.052,0.0
5587,Sungai Selangor Fasa 3 Stage 1,6.25,2022-12-28,91.10,37.33,1.68,0.13,0.14,0.0,0.0,0.0,0.0,0.0,12,0.0,0.0,0.0,0.0,0.0,12.75,0.0,5.30,0.0,0.0,0.0,47.56,0.053,0.0
5588,Sungai Selangor Fasa 3 Stage 1,6.30,2022-12-29,63.48,38.67,1.73,0.12,0.13,0.0,0.0,0.0,0.0,0.0,12,0.0,0.0,0.0,0.0,0.0,12.67,0.0,5.42,0.0,0.0,0.0,45.46,0.040,0.0
5589,Sungai Selangor Fasa 3 Stage 1,6.33,2022-12-30,66.59,44.08,1.72,0.10,0.17,0.0,0.0,0.0,0.0,0.0,12,0.0,0.0,0.0,0.0,0.0,12.40,0.0,5.25,0.0,0.0,0.0,40.45,0.046,0.0


In [31]:
jam4['Soda_Ash'] = jam4.apply(lambda row: row['Soda_Ash_T'] + row['Soda Ash'], axis=1)
jam4['Liquid_Chlorine'] = jam4.apply(lambda row: row['Liquid_Chlorine_T'] + row['Liquid Chlorine'], axis=1)
jam4['Liquid_Polyaluminium_Chloride'] = jam4.apply(lambda row: row['Liquid Polyaluminium Chloride (mg/L)'] + row['Liquid Polyaluminium Chloride'], axis=1)
jam4['Hydrated_Lime'] = jam4.apply(lambda row: row['Hydrated Lime'] + row['Hydrated_Lime_Pow_T'], axis=1)
jam4['Aluminium_Sulphate'] = jam4.apply(lambda row: row['Liquid Aluminium Sulphate'] + row['Kibbled Aluminium Sulphate'], axis=1)
jam4['Sodium_Silico_Fluoride'] = jam4.apply(lambda row: row['Sodium Silico Fluoride'] + row['Sodium Silico Fluoride (mg/L)'], axis=1)

jam4 = jam4.drop(['Sodium Aluminosilicate','Flopam FA 920 PWG','Raw Water Polyelectrolyte 4290 PWG / 4190 PWG / AN910 (mg/L)','Flopam FO 4290 PWG','Sodium_Silico_Fluoride','Microsand','Sodium Silico Fluoride (mg/L)','Sodium Silico Fluoride','Kibbled Aluminium Sulphate','Liquid Aluminium Sulphate','Hydrated Lime','Hydrated_Lime_Pow_T','Soda_Ash_T','Soda Ash','Liquid_Chlorine_T','Liquid Chlorine','Liquid Polyaluminium Chloride (mg/L)','Liquid Polyaluminium Chloride'],axis = 1)

In [32]:
## Feature Engineering (Daily % Changes compared to median value)
the_name = jam4.drop(['wtp', 'Date','Month'],axis=1).columns
for i in the_name:
    means = jam4.groupby('wtp')[f'{i}'].transform('median')
    jam4[f'{i}_chpctm'] = (jam4[f'{i}'] - means)/means*100

In [33]:
jam4 = jam4.drop(['pH_avg', 'Turbidity_avg', 'Color_avg', 'Iron_avg',
       'Ammonia_avg', 'Manganese_avg', 'Month', 'Soda_Ash', 'Liquid_Chlorine',
       'Liquid_Polyaluminium_Chloride', 'Hydrated_Lime', 'Aluminium_Sulphate'],axis = 1)

In [34]:
cols_to_change = [col for col in jam4.columns if '_chpctm' in col]

# create a dictionary of old column names to new column names
new_cols = {col: col.replace('_chpctm', '') for col in cols_to_change}

# rename the columns in the DataFrame
jam4 = jam4.rename(columns=new_cols)
jam4 = jam4.replace([np.inf], 9999 )
jam4 = jam4.replace([-np.inf], -9999 )

In [35]:
jam4.to_csv('CP_data\ChemPare_SgSel_4.csv')

In [36]:
tfi = [x for x in jam4.columns if "_chpctm" in x]
to_c4 = jam4[tfi].copy()

## Dropping Auxiliary Chemica, Polyelectrolyte & Flouridation because project does not focus study on these product (Low usage & only certain WTP using it)
#to_c4 = to_c4.drop(['Auxiliary Chemical_chpctm', 'Polyelectrolyte_chpctm','Flouridation_chpctm'],axis=1)

In [37]:
jam4

,wtp,Date,pH_avg,Turbidity_avg,Color_avg,Iron_avg,Ammonia_avg,Manganese_avg,Soda_Ash,Liquid_Chlorine,Liquid_Polyaluminium_Chloride,Hydrated_Lime,Aluminium_Sulphate
0,Batang Kali,2021-04-01,0.594354,246.385542,1610.344828,30.000000,180.0,0.000000,NaN,NaN,NaN,NaN,NaN
1,Batang Kali,2021-04-02,-1.188707,-31.686747,44.827586,10.000000,20.0,0.000000,NaN,NaN,NaN,NaN,NaN
2,Batang Kali,2021-04-03,-1.783061,77.349398,44.827586,-35.000000,120.0,0.000000,NaN,NaN,NaN,NaN,NaN
3,Batang Kali,2021-04-04,-1.931649,14.457831,1079.310345,10.000000,370.0,100.000000,NaN,NaN,NaN,NaN,NaN
4,Batang Kali,2021-04-05,-1.783061,8.554217,-41.379310,-30.000000,50.0,0.000000,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2999,Sungai Selangor Fasa 3,2021-09-26,-2.426343,178.072740,2.196721,89.808917,70.0,109.090909,NaN,NaN,NaN,NaN,NaN
3000,Sungai Selangor Fasa 3,2021-09-27,-4.332756,129.598877,4.655738,68.152866,50.0,150.000000,NaN,NaN,NaN,NaN,NaN
3001,Sungai Selangor Fasa 3,2021-09-28,-3.986135,125.026313,5.737705,20.382166,100.0,72.727273,NaN,NaN,NaN,NaN,NaN
3002,Sungai Selangor Fasa 3,2021-09-29,-3.986135,208.057537,-14.754098,67.515924,140.0,159.090909,NaN,NaN,NaN,NaN,NaN


In [102]:
jam4.columns

Index(['wtp', 'pH_avg', 'Date', 'Turbidity_avg', 'Color_avg', 'Iron_avg',
       'Ammonia_avg', 'Manganese_avg', 'Month', 'Soda_Ash', 'Liquid_Chlorine',
       'Liquid_Polyaluminium_Chloride', 'Hydrated_Lime', 'Aluminium_Sulphate',
       'pH_avg_chpctm', 'Turbidity_avg_chpctm', 'Color_avg_chpctm',
       'Iron_avg_chpctm', 'Ammonia_avg_chpctm', 'Manganese_avg_chpctm',
       'Soda_Ash_chpctm', 'Liquid_Chlorine_chpctm',
       'Liquid_Polyaluminium_Chloride_chpctm', 'Hydrated_Lime_chpctm',
       'Aluminium_Sulphate_chpctm'],
      dtype='object')

In [111]:
## General Columns for EDA
ge = ['pH_avg','Turbidity_avg', 'Color_avg','Iron_avg', 'Ammonia_avg', 'Manganese_avg','Soda_Ash', 'Liquid_Chlorine',
       'Liquid_Polyaluminium_Chloride', 'Hydrated_Lime', 'Aluminium_Sulphate']
lib = list(to_c4.columns) + ['Date','wtp'] + ge
to_c6 = FFDB[lib].copy()

KeyError: "['pH_avg_chpctm', 'Turbidity_avg_chpctm', 'Color_avg_chpctm', 'Iron_avg_chpctm', 'Ammonia_avg_chpctm', 'Manganese_avg_chpctm', 'Soda_Ash_chpctm', 'Liquid_Chlorine_chpctm', 'Liquid_Polyaluminium_Chloride_chpctm', 'Hydrated_Lime_chpctm', 'Aluminium_Sulphate_chpctm', 'Soda_Ash', 'Liquid_Chlorine', 'Liquid_Polyaluminium_Chloride', 'Hydrated_Lime', 'Aluminium_Sulphate'] not in index"

In [79]:
to_c6.to_csv('C_data/WGIDB_SgSel.csv')